## Align utterance in Memor to open subtitle parallel bitext

In [14]:
import json
import os

from lxml import etree
import lxml
from tqdm import tqdm
import pickle as pkl
import re
import json
import os

#### Load Friends Dataset

In [77]:
# Load Open Subtitle Bitext
xml_file = etree.parse('../open_subtitle/en-zh_cn.tmx')
# Collect subtitles
en_subtitle = []
zh_subtitle = []
root = xml_file.getroot()
count = 0
for node in tqdm(root.iter()):
    if node.tag=='seg':
        if count % 2 ==0:
            en_subtitle.append(node.text)
        else:
            zh_subtitle.append(node.text)
        count += 1

49845028it [00:58, 852163.01it/s] 


In [32]:
with open('emory_nlp_dataset/json/friends_season_01.json') as f:
    data = json.load(f)

In [63]:

episodes = data['episodes']
print(type(episodes), len(episodes))
temp = episodes[0]
output = {}
for episode in episodes:
    transcripts = []
    episode_id = episode['episode_id']
    print(episode_id)
    scenes = episode['scenes']
    for scene in scenes:
        utterances = scene['utterances']
        for utt in utterances:
            transcript = utt['transcript'].strip()
            transcripts.append(transcript)
    output[episode_id] = transcripts

<class 'list'> 25
s03_e01
s03_e02
s03_e03
s03_e04
s03_e05
s03_e06
s03_e07
s03_e08
s03_e09
s03_e10
s03_e11
s03_e12
s03_e13
s03_e14
s03_e15
s03_e16
s03_e17
s03_e18
s03_e19
s03_e20
s03_e21
s03_e22
s03_e23
s03_e24
s03_e25


In [71]:
# Load Input Data
dir_path = 'emory_nlp_dataset/json/'
output = {}
for file_name in os.listdir(dir_path):
    # Load input data
    input_path = dir_path + file_name
    output_path = 'emory_nlp_dataset/pkl/friends_' + file_name.strip().split('_')[-1].split('.')[0] + '.pkl'
    with open(input_path) as f:
        data = json.load(f)
    episodes = data['episodes']
    print(type(episodes), len(episodes))
    temp = episodes[0]
    for episode in episodes:
        transcripts = []
        episode_id = episode['episode_id']
        print(episode_id)
        scenes = episode['scenes']
        for scene in scenes:
            utterances = scene['utterances']
            for utt in utterances:
                transcript = utt['transcript'].strip()
                transcripts.append(transcript)
        output[episode_id] = transcripts
data = output
# pkl.dump(" ", open('', 'wb'))

<class 'list'> 24
s01_e01
s01_e02
s01_e03
s01_e04
s01_e05
s01_e06
s01_e07
s01_e08
s01_e09
s01_e10
s01_e11
s01_e12
s01_e13
s01_e14
s01_e15
s01_e16
s01_e17
s01_e18
s01_e19
s01_e20
s01_e21
s01_e22
s01_e23
s01_e24
<class 'list'> 25
s06_e01
s06_e02
s06_e03
s06_e04
s06_e05
s06_e06
s06_e07
s06_e08
s06_e09
s06_e10
s06_e11
s06_e12
s06_e13
s06_e14
s06_e15
s06_e16
s06_e17
s06_e18
s06_e19
s06_e20
s06_e21
s06_e22
s06_e23
s06_e24
s06_e25
<class 'list'> 18
s10_e01
s10_e02
s10_e03
s10_e04
s10_e05
s10_e06
s10_e07
s10_e08
s10_e09
s10_e10
s10_e11
s10_e12
s10_e13
s10_e14
s10_e15
s10_e16
s10_e17
s10_e18
<class 'list'> 24
s07_e01
s07_e02
s07_e03
s07_e04
s07_e05
s07_e06
s07_e07
s07_e08
s07_e09
s07_e10
s07_e11
s07_e12
s07_e13
s07_e14
s07_e15
s07_e16
s07_e17
s07_e18
s07_e19
s07_e20
s07_e21
s07_e22
s07_e23
s07_e24
<class 'list'> 24
s04_e01
s04_e02
s04_e03
s04_e04
s04_e05
s04_e06
s04_e07
s04_e08
s04_e09
s04_e10
s04_e11
s04_e12
s04_e13
s04_e14
s04_e15
s04_e16
s04_e17
s04_e18
s04_e19
s04_e20
s04_e21
s04_e22
s04_e2

In [72]:
print(data['s01_e02'])

["What you guys don't understand is, for us, kissing is as important as any part of it.", "Yeah, right!.......Y'serious?", 'Oh, yeah!', 'Everything you need to know is in that first kiss.', 'Absolutely.', "Yeah, I think for us, kissing is pretty much like an opening act, y'know? I mean it's like the stand-up comedian you have to sit through before Pink Floyd comes out.", "Yeah, and-and it's not that we don't like the comedian, it's that-that... that's not why we bought the ticket.", "The problem is, though, after the concert's over, no matter how great the show was, you girls are always looking for the comedian again, y'know? I mean, we're in the car, we're fighting traffic... basically just trying to stay awake.", "Yeah, well, word of advice: Bring back the comedian. Otherwise next time you're gonna find yourself sitting at home, listening to that album alone.", '....Are we still talking about sex?', "No, it's good, it is good, it's just that- mm- doesn't she seem a little angry?", 'W

In [73]:
def get_segments(episode_key, all_data):
    # Gather utteranes the defined episode
    segments = []
    pattern = r'\.|\,|\?|\!|\;'
    for utt in all_data[episode_key]:
        for item in re.split(pattern, utt):
                    tokens = item.strip().split(' ')
                    length = len(tokens)
                    num_iter = length // 6
                    for i in range(num_iter):
                        segments.append(" ".join(tokens[i*6: i*6+6]))
    return segments

In [76]:
results = get_segments('s01_e07', data)
for x in results:
    print(x)

Central Perk is proud to present
the music of Miss Phoebe Buffay
I want to start with a
song thats about that moment when
you suddenly realize what life is
The entire city is blacked out
Mom says it's all of Manhattan
and they have no idea when
Who am I gonna meet in
Could we talk about this later
I want to call my apartment
I am trapped in an ATM
that is the part to focus
I'm just stuck at the bank
I saw you with some guy
and these are the only candles
Ugly Naked Guy lit a bunch
It's been fourteen and a half
minutes and you still have not
Would you like to call somebody
about 300 guys I went to
I'm trppd in an ATM vstbl
I have no idea what you
He's trapped in an ATM vestibule
like that thought never entered my
my weirdest place would have to
the women's room on the second
floor of the New York CIty
What were you doing in a
'It's a Small World After All
Carol and I went behind a
couple of those mechanical Dutch children
and we were asked never to
The weirdest place would have to
I just

In [78]:
episode_keys = tuple(data.keys())
print(episode_keys)

('s01_e01', 's01_e02', 's01_e03', 's01_e04', 's01_e05', 's01_e06', 's01_e07', 's01_e08', 's01_e09', 's01_e10', 's01_e11', 's01_e12', 's01_e13', 's01_e14', 's01_e15', 's01_e16', 's01_e17', 's01_e18', 's01_e19', 's01_e20', 's01_e21', 's01_e22', 's01_e23', 's01_e24', 's06_e01', 's06_e02', 's06_e03', 's06_e04', 's06_e05', 's06_e06', 's06_e07', 's06_e08', 's06_e09', 's06_e10', 's06_e11', 's06_e12', 's06_e13', 's06_e14', 's06_e15', 's06_e16', 's06_e17', 's06_e18', 's06_e19', 's06_e20', 's06_e21', 's06_e22', 's06_e23', 's06_e24', 's06_e25', 's10_e01', 's10_e02', 's10_e03', 's10_e04', 's10_e05', 's10_e06', 's10_e07', 's10_e08', 's10_e09', 's10_e10', 's10_e11', 's10_e12', 's10_e13', 's10_e14', 's10_e15', 's10_e16', 's10_e17', 's10_e18', 's07_e01', 's07_e02', 's07_e03', 's07_e04', 's07_e05', 's07_e06', 's07_e07', 's07_e08', 's07_e09', 's07_e10', 's07_e11', 's07_e12', 's07_e13', 's07_e14', 's07_e15', 's07_e16', 's07_e17', 's07_e18', 's07_e19', 's07_e20', 's07_e21', 's07_e22', 's07_e23', 's07_e24'

In [79]:
episode_indexs = {}
for item in tqdm(episode_keys[:2]):
    temp = []
    segments = get_segments(item, data)
    for segment in tqdm(segments):
        for i, subtitle in enumerate(en_subtitle):
            if segment in subtitle:
                temp.append([i, segment, en_subtitle[i], zh_subtitle[i]])
    episode_indexs[item] = temp
pkl.dump(episode_indexs, open('episode_indexs.pkl', 'wb'))

100%|██████████| 2/2 [08:22<00:00, 251.39s/it]


In [84]:
temp = pkl.load(open('episode_indexs.pkl', 'rb'))

for x in temp:
    print(x)
    for item in temp[x]:
        print(item)
    print('=='*50)

s01_e01
[3590843, "There's gotta be something wrong with", "There's gotta be something wrong with him.", '他一定有什么毛病.']
[1664584, 'So does he have a hump', 'So does he have a hump and a hairpiece?', '他驼背？ 既驼背又带假发？']
[451073, "I don't want her to go", "I don't want her to go to a zoo. I want to set her free.", '我不想让她去动物园,我想让她自由']
[944537, "I don't want her to go", "I don't want her to go away.", '我不会和她分开的 听清楚了吗？ 医生']
[1664586, "I don't want her to go", "I don't want her to go through what I did with Carl.", '我只是不想你重蹈我和卡尔的覆辙']
[1854679, "I don't want her to go", "I don't want her to go.", '我觉得\ue4d1不想让她走']
[3490867, "I don't want her to go", "I don't even want to go. I don't want her to go.", '我什至都不想去 我不想让她去']
[3964609, "I don't want her to go", "I don't want her to go on.", '我不想让她再继续了']
[1664587, 'This is not even a date', 'Okay, everybody relax. This is not even a date.', '各位别急， 这不算约会']
[1664588, "It's just two people going out", "It's just two people going out to dinner and not having s